<a href="https://colab.research.google.com/github/alinaalborova/russian_idioms_processing/blob/main/MICE_fastText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Idiom Type and Token Classification

Based on [MICE: Mining Idioms with Contextual Embeddings](https://arxiv.org/pdf/2008.05759.pdf) by  Škvorc et al.


In [ ]:
!pip install transformers
!pip install tensor2tensor

     |████████████████████████████████| 2.3MB 4.1MB/s 
     |████████████████████████████████| 901kB 41.2MB/s 
     |████████████████████████████████| 3.3MB 35.3MB/s 
     |████████████████████████████████| 1.5MB 3.9MB/s 
     |████████████████████████████████| 655kB 37.5MB/s 
     |████████████████████████████████| 983kB 27.5MB/s 
     |████████████████████████████████| 368kB 37.5MB/s 
     |████████████████████████████████| 368kB 37.5MB/s 
     |████████████████████████████████| 194kB 38.0MB/s 
     |████████████████████████████████| 81kB 8.2MB/s 
     |████████████████████████████████| 358kB 27.9MB/s 
     |████████████████████████████████| 686kB 36.0MB/s 
     |████████████████████████████████| 5.6MB 27.9MB/s 
     |████████████████████████████████| 256kB 40.7MB/s 
  Created wheel for pypng: filename=pypng-0.0.20-cp37-none-any.whl size=67163 sha256=dab200541f1199f8a657303d4689ef9119a07f64e98de14b95c973a68fb18515
  Stored in directory: /root/.cache/pip/wheels/41/6b/ef/0493b536b6d472

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import tensorflow as tf
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Activation, TimeDistributed, Masking, GRU
from tensorflow.keras import Sequential
from tensorflow.keras.utils import to_categorical
from ast import literal_eval
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from collections import Counter
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score

## Install & Import fastText

In [ ]:
pip install fasttext

     |████████████████████████████████| 71kB 2.6MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3098404 sha256=8f73f118b52ba2039ff7f23613112ddddb049c9ca0bcb4a91f4a3bdc8111abff
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [ ]:
import fasttext.util
fasttext.util.download_model('ru', if_exists='ignore')  # Russian
ft = fasttext.load_model('cc.ru.300.bin')

## Dataset

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
dataset_vnc_dir = '/content/drive/MyDrive/ВКР/Sense Disambiguation Corpus/VNCs_Annotated.csv'
data_vnc = pd.read_csv(dataset_vnc_dir )
data_vnc.head(2)

,Idiom Normal,Idiom Inflected,Label,Example
0,бить карту,бил карту,0,Он бил карту за картой и загребал золото и кре...
1,бить карту,бил карту,0,"Ермолов держал карты, сощуря правый глаз; ког..."


In [ ]:
data_vnc.Label.value_counts()

1    455
0    438
Name: Label, dtype: int64

In [ ]:
data_vnc.shape

(893, 4)

In [ ]:
len(data_vnc['Idiom Normal'].value_counts())

51

In [ ]:
dataset_anc_dir = '/content/drive/MyDrive/ВКР/Sense Disambiguation Corpus/ANCs_Annotated.csv'
data_anc = pd.read_csv(dataset_anc_dir )
data_anc.head(2)

,Idiom Normal,Idiom Inflected,Label,Example
0,избитая дорога,избитой дороге,0,"С бурной быстротой, возможной только в сновид..."
1,избитая дорога,избитой дороге,0,"Как почтовый возок на избитой дороге, прыгает..."


In [ ]:
len(data_anc['Idiom Inflected'].value_counts())

180

In [ ]:
data_anc['Idiom Normal'].value_counts()

больное место                 57
правая рука                   56
болевая точка                 52
нож острый                    49
путеводная звезда             48
лавровый венок                44
бедный родственник            41
тяжёлая рука                  38
зелёная улица                 38
ваш брат                      34
вавилонское столпотворение    30
наша сестра                   28
пороховая бочка               27
дальний прицел                25
вторая ступень                23
заблудшая овца                23
старый воробей                22
красная бумажка               21
синяя птица                   20
долгая песня                  18
другой разговор               18
старый гриб                   16
чёрная кость                  15
девичья кожа                  12
маковое зерно                 10
музейная редкость             10
избитая дорога                10
ободранная кошка               9
куриная голова                 9
чернильная строка              3
Name: Idio

In [ ]:
len(data_anc['Idiom Normal'].value_counts())

30

In [ ]:
data = pd.concat([data_vnc, data_anc], ignore_index=True)
data.head(2)

,Idiom Normal,Idiom Inflected,Label,Example
0,бить карту,бил карту,0,Он бил карту за картой и загребал золото и кре...
1,бить карту,бил карту,0,"Ермолов держал карты, сощуря правый глаз; ког..."


In [ ]:
data['Label'].value_counts()

0    856
1    843
Name: Label, dtype: int64

In [ ]:
data.shape

(1699, 4)

In [ ]:
data.tail(2)

,Idiom Normal,Idiom Inflected,Label,Example
1697,бедный родственник,бедными родственниками,0,"Проходя мимо церквей, я вижу иногда человека,..."
1698,бедный родственник,бедных родственниках,0,"[Егор Дмитрич Глумов, муж] У молодой женщин..."


In [ ]:
data.shape

(1699, 4)

## Tokenize & Get fastText Embeddings

In [ ]:
import re
GROUPING_SPACE_REGEX = re.compile('([^\w_-]|[+])', re.U)

def tokenize(text):
  """
  Split text into tokens. Don't split by hyphen.
  """
  return [t for t in GROUPING_SPACE_REGEX.split(text)
          if t and not t.isspace()]

from keras.preprocessing.sequence import pad_sequences

def pad_sentence(tokenized, max_len):
  print('\nPadding/truncating all sentences to %d values...' % max_len)
  print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

  # Pad our input tokens with value 0.
  # "post" indicates that we want to pad and truncate at the end of the sequence,
  # as opposed to the beginning.
  input_ids = pad_sequences(tokenized, maxlen=max_len, dtype="long", 
                            value=0, truncating="post", padding="post")
  print('\nDone.')
  return input_ids

In [ ]:
def get_embeddings(text, length, padded_size=100):
  vec_size = 300
  embeddings_list = []
  for i, sentence in enumerate(text):
    local_embeddings = np.empty(shape=(padded_size, ), dtype=object)
    for id, token in enumerate(local_embeddings):
      try:
        local_embeddings[id] = ft.get_word_vector(sentence[id])
      except:
        local_embeddings[id] = np.zeros(vec_size)
    embeddings_list.append(local_embeddings)
  return embeddings_list

### All

In [ ]:
contexts_tokenized_all = data.Example.apply(tokenize)
embedded_contexts_all = get_embeddings(contexts_tokenized_all, length=len(list(contexts_tokenized_all)))

In [ ]:
labels = to_categorical(data.Label)
X_train, X_test = train_test_split(embedded_contexts_all, test_size=0.2)
Y_train, Y_test = train_test_split(labels, test_size=0.2)
X_train = tf.convert_to_tensor(X_train)
X_test = tf.convert_to_tensor(X_test)

### VNC

In [ ]:
contexts_tokenized_vnc = data_vnc.Example.apply(tokenize)
embedded_contexts_vnc = get_embeddings(contexts_tokenized_vnc, length=len(list(contexts_tokenized_vnc)))

In [ ]:
labels_vnc = to_categorical(data_vnc.Label)
X_train_vnc, X_test_vnc = train_test_split(embedded_contexts_vnc, test_size=0.2)
Y_train_vnc, Y_test_vnc = train_test_split(labels_vnc, test_size=0.2)
X_train_vnc = tf.convert_to_tensor(X_train_vnc)
X_test_vnc = tf.convert_to_tensor(X_test_vnc)

### ANC

In [ ]:
contexts_tokenized_anc = data_anc.Example.apply(tokenize)
embedded_contexts_anc = get_embeddings(contexts_tokenized_anc, length=len(list(contexts_tokenized_anc)))

In [ ]:
labels_anc = to_categorical(data_anc.Label)
X_train_anc, X_test_anc = train_test_split(embedded_contexts_anc, test_size=0.2)
Y_train_anc, Y_test_anc = train_test_split(labels_anc, test_size=0.2)
X_train_anc = tf.convert_to_tensor(X_train_anc)
X_test_anc = tf.convert_to_tensor(X_test_anc)

## RNN

In [ ]:
MAX_SEQUENCE_LEN = 500
VECTOR_DIM = 300
NUM_CLASSES = 2

In [ ]:
def build_model():
  model = Sequential()
  model.add(Masking(mask_value=0., input_shape=(MAX_SEQUENCE_LEN,VECTOR_DIM)))
  forward_layer = GRU(10, return_sequences=False, dropout=0.5)
  backward_layer = GRU(10, return_sequences=False, dropout=0.5,
                      go_backwards=True)
  model.add(Bidirectional(forward_layer, backward_layer=backward_layer,
                        input_shape=(MAX_SEQUENCE_LEN,VECTOR_DIM)))
  model.add(Dense(NUM_CLASSES))
  model.add(Activation('softmax'))

  model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
  print('compiled model')
  return model

### All

In [ ]:
model_all = build_model()

compiled model


In [ ]:
model_all = build_model()
model_all.fit(X_train, Y_train, batch_size=8, epochs=10)

compiled model
Epoch 1/10
170/170 [==============================] - 23s 92ms/step - loss: 0.6952 - accuracy: 0.5173
Epoch 2/10
170/170 [==============================] - 16s 92ms/step - loss: 0.6917 - accuracy: 0.5202
Epoch 3/10
170/170 [==============================] - 16s 92ms/step - loss: 0.6874 - accuracy: 0.5423
Epoch 4/10
170/170 [==============================] - 15s 91ms/step - loss: 0.6862 - accuracy: 0.5570
Epoch 5/10
170/170 [==============================] - 16s 92ms/step - loss: 0.6805 - accuracy: 0.5681
Epoch 6/10
170/170 [==============================] - 16s 93ms/step - loss: 0.6720 - accuracy: 0.6026
Epoch 7/10
170/170 [==============================] - 15s 91ms/step - loss: 0.6743 - accuracy: 0.5651
Epoch 8/10
170/170 [==============================] - 15s 91ms/step - loss: 0.6668 - accuracy: 0.5857
Epoch 9/10
170/170 [==============================] - 16s 92ms/step - loss: 0.6603 - accuracy: 0.6115
Epoch 10/10
170/170 [==============================] - 16s 91ms/ste

In [ ]:
model_all.evaluate(X_test, Y_test)

11/11 [==============================] - 3s 29ms/step - loss: 0.7097 - accuracy: 0.4706


[0.7096795439720154, 0.47058823704719543]

In [ ]:
preds_all = model_all.predict(np.asarray(X_test))
f1_score(np.argmax(preds_all, axis=1), np.argmax(Y_test, axis=1))

0.5027624309392265

### VNC

In [ ]:
model_vnc = build_model()
model_vnc.fit(X_train_vnc, Y_train_vnc, batch_size=8, epochs=10)

compiled model
Epoch 1/10
90/90 [==============================] - 15s 91ms/step - loss: 0.6948 - accuracy: 0.5154
Epoch 2/10
90/90 [==============================] - 8s 91ms/step - loss: 0.6886 - accuracy: 0.5378
Epoch 3/10
90/90 [==============================] - 8s 91ms/step - loss: 0.6822 - accuracy: 0.5588
Epoch 4/10
90/90 [==============================] - 8s 90ms/step - loss: 0.6777 - accuracy: 0.5826
Epoch 5/10
90/90 [==============================] - 8s 93ms/step - loss: 0.6735 - accuracy: 0.6022
Epoch 6/10
90/90 [==============================] - 8s 91ms/step - loss: 0.6551 - accuracy: 0.6303
Epoch 7/10
90/90 [==============================] - 8s 92ms/step - loss: 0.6503 - accuracy: 0.6499
Epoch 8/10
90/90 [==============================] - 8s 92ms/step - loss: 0.6441 - accuracy: 0.6317
Epoch 9/10
90/90 [==============================] - 8s 90ms/step - loss: 0.6329 - accuracy: 0.6429
Epoch 10/10
90/90 [==============================] - 8s 90ms/step - loss: 0.6173 - accuracy: 

In [ ]:
model_vnc.evaluate(X_test_vnc, Y_test_vnc)

6/6 [==============================] - 2s 29ms/step - loss: 0.7380 - accuracy: 0.4860


[0.7380333542823792, 0.48603352904319763]

In [ ]:
preds_vnc = model_vnc.predict(np.array(X_test_vnc))
f1_score(np.argmax(preds_vnc, axis=1), np.argmax(Y_test_vnc, axis=1))

0.5533980582524272

### ANC

In [ ]:
model_anc = build_model()
model_anc.fit(X_train_anc, Y_train_anc, batch_size=8, epochs=10)

compiled model
Epoch 1/10
81/81 [==============================] - 14s 91ms/step - loss: 0.6981 - accuracy: 0.4984
Epoch 2/10
81/81 [==============================] - 8s 93ms/step - loss: 0.6895 - accuracy: 0.5435
Epoch 3/10
81/81 [==============================] - 7s 91ms/step - loss: 0.6846 - accuracy: 0.5466
Epoch 4/10
81/81 [==============================] - 8s 94ms/step - loss: 0.6805 - accuracy: 0.5652
Epoch 5/10
81/81 [==============================] - 7s 91ms/step - loss: 0.6767 - accuracy: 0.5854
Epoch 6/10
81/81 [==============================] - 7s 91ms/step - loss: 0.6681 - accuracy: 0.6071
Epoch 7/10
81/81 [==============================] - 7s 90ms/step - loss: 0.6599 - accuracy: 0.6304
Epoch 8/10
81/81 [==============================] - 8s 93ms/step - loss: 0.6564 - accuracy: 0.6118
Epoch 9/10
81/81 [==============================] - 7s 93ms/step - loss: 0.6444 - accuracy: 0.6568
Epoch 10/10
81/81 [==============================] - 8s 94ms/step - loss: 0.6351 - accuracy: 

In [ ]:
model_anc.evaluate(X_test_anc, Y_test_anc)

6/6 [==============================] - 3s 25ms/step - loss: 0.6913 - accuracy: 0.5309


[0.6913085579872131, 0.5308641791343689]

In [ ]:
preds_anc = model_anc.predict(X_test_anc)
f1_score(np.argmax(preds_anc, axis=1), np.argmax(Y_test_anc, axis=1))

0.5189873417721519